### Step 1: 读取数据

In [1]:
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

SUMMARY_DIR = "/log/mnist-log"
if tf.gfile.Exists(SUMMARY_DIR):
    tf.gfile.DeleteRecursively(SUMMARY_DIR)
tf.gfile.MakeDirs(SUMMARY_DIR)

mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

print "Training data size: ", mnist.train.num_examples
print "Validating data size: ", mnist.validation.num_examples
print "Testing data size: ", mnist.test.num_examples

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Training data size:  55000
Validating data size:  5000
Testing data size:  10000


### Step 2: 创建神经网络，并指定log信息

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1, seed = 2)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read, and
    adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights, layer_name + '/weights')
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases, layer_name + '/biases')
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.histogram_summary(layer_name + '/pre_activations', preactivate)
        activations = act(preactivate, 'activation')
        tf.histogram_summary(layer_name + '/activations', activations)
        return activations

# Create a multilayer model.
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.image_summary('input', image_shaped_input, 20)
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

hidden_nodes = 500
hidden1 = nn_layer(x, 784, hidden_nodes, 'layer1')
y = nn_layer(hidden1, hidden_nodes, 10, 'layer2', act=tf.nn.softmax)
print("Network created!")

Network created!


### Step 3: 指定训练过程

In [3]:
with tf.name_scope('cross_entropy'):
    diff = y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
    with tf.name_scope('total'):
        cross_entropy = -tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.scalar_summary('accuracy', accuracy)

print("Training & eval step setup!")

Training & eval step setup!


### Step 4: 指定日志文件地址， 初始化所有变量

In [4]:
# Merge all the summaries and write them out to　SUMMARY_DIR
merged = tf.merge_all_summaries()
train_writer = tf.train.SummaryWriter(SUMMARY_DIR + '/train', sess.graph)
test_writer = tf.train.SummaryWriter(SUMMARY_DIR + '/test')

tf.initialize_all_variables().run()

validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
test_feed = {x: mnist.test.images, y_: mnist.test.labels}
print("Log init done!")

Log init done!


### Step 5: 训练模型

In [5]:
def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
        xs, ys = mnist.train.next_batch(100)
    else:
        xs, ys = mnist.test.images, mnist.test.labels
    return {x: xs, y_: ys}

STEPS = 500
saver = tf.train.Saver()
time_begin = time.time()
print("Training begins @ %f" % time_begin)
for i in range(STEPS):
    _, summary = sess.run([train_step, merged], feed_dict=feed_dict(True))

    if i % 100 == 0:
        # Write summary
        train_writer.add_summary(summary, i)
        
        summary = sess.run(merged, feed_dict=feed_dict(False))
        test_writer.add_summary(summary, i)        
        
        # Print training information.
        validate_acc = sess.run(accuracy, feed_dict=validate_feed)
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training step(s), validation accuracy = %g, test accuracy = %g" %  
              (i, validate_acc, test_acc))
        
        # Store model.
        if i == 300: saver.save(sess, "/tmp/saved_model")

            
time_end = time.time()
print("Training ends @ %f" % time_end)
training_time = time_end - time_begin
print("Training elapsed time: %f s" % training_time)
test_acc = sess.run(accuracy, feed_dict=test_feed)
print("After %d training step(s), test accuracy = %g" %  (STEPS, test_acc))

Training begins @ 1466980873.216786
After 0 training step(s), validation accuracy = 0.2188, test accuracy = 0.2096
After 100 training step(s), validation accuracy = 0.9168, test accuracy = 0.9113
After 200 training step(s), validation accuracy = 0.9382, test accuracy = 0.934
After 300 training step(s), validation accuracy = 0.9528, test accuracy = 0.9483
After 400 training step(s), validation accuracy = 0.9538, test accuracy = 0.9535
Training ends @ 1466980891.376311
Training elapsed time: 18.159525 s
After 500 training step(s), test accuracy = 0.9557


In [6]:
saver.restore(sess, "/tmp/saved_model")
test_acc = sess.run(accuracy, feed_dict=test_feed)
print("Test accuracy for stored model = %g" %  (test_acc))

sess.close()

Test accuracy for stored model = 0.9483
